In [131]:
#import libraries
import os
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [132]:
sports_data_21=pd.read_csv('/content/drive/MyDrive/MIDSEM PROJECT/players_21.csv')#train

In [133]:
sports_data_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


Drop features with 30% NaN

In [134]:
threshold = 0.30 * len(sports_data_21)

# Identify columns with 30% or more NaN values
columns_with_nan = sports_data_21.columns[sports_data_21.isna().sum() >= threshold]

# Drop the identified columns
sports_data_21.drop(columns=columns_with_nan, inplace=True)

Getting numerical and object columns

In [135]:
numerical_columns = sports_data_21.select_dtypes(include=['int64', 'float64'])

In [136]:
object_columns = sports_data_21.select_dtypes(include=['object']).columns

In [137]:
object_columns

Index(['player_url', 'short_name', 'long_name', 'player_positions', 'dob',
       'club_name', 'league_name', 'club_position', 'club_joined',
       'nationality_name', 'preferred_foot', 'work_rate', 'body_type',
       'real_face', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam',
       'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm',
       'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url',
       'club_logo_url', 'club_flag_url', 'nation_flag_url'],
      dtype='object')

Encoding object columns

In [138]:
for column in object_columns:
    sports_data_21[column] = pd.factorize(sports_data_21[column])[0]


Imputing  columns

In [139]:
# Create a SimpleImputer instance with your chosen strategy
imputer = SimpleImputer(strategy='mean')

# Iterate over columns and impute
for column in sports_data_21:
    sports_data_21[column] = imputer.fit_transform(sports_data_21[[column]])


In [140]:
sports_data_21.isna().any() #Check if there are any NaNs

sofifa_id           False
player_url          False
short_name          False
long_name           False
player_positions    False
                    ...  
gk                  False
player_face_url     False
club_logo_url       False
club_flag_url       False
nation_flag_url     False
Length: 102, dtype: bool

In [141]:
c = sports_data_21.corr()
c["overall"].sort_values(ascending=False)

overall                1.000000
movement_reactions     0.867234
mentality_composure    0.705252
passing                0.662090
potential              0.636366
                         ...   
dob                   -0.595900
short_name            -0.923514
long_name             -0.974388
player_face_url       -0.975045
player_url            -0.975045
Name: overall, Length: 102, dtype: float64

In [142]:
#Using correlation values greater than 0.4 we selected these
s =['overall','movement_reactions','passing','mentality_composure','dribbling','release_clause_eur','wage_eur','value_eur','power_shot_power','physic','mentality_vision',
           'attacking_short_passing','shooting','skill_long_passing','age','skill_ball_control','international_reputation','skill_curve','attacking_crossing']

In [143]:
#We picked the top 12
selected =['overall','movement_reactions','passing','dribbling','mentality_composure','power_shot_power','attacking_short_passing','shooting','age','physic','mentality_vision','international_reputation']

In [144]:
sports_data_21 = sports_data_21[selected]

In [145]:
sports_data_21.head()

,overall,movement_reactions,passing,dribbling,mentality_composure,power_shot_power,attacking_short_passing,shooting,age,physic,mentality_vision,international_reputation
0,93.0,94.0,91.0,95.0,96.0,86.0,91.0,92.0,33.0,65.0,95.0,5.0
1,92.0,95.0,81.0,89.0,95.0,94.0,82.0,93.0,35.0,77.0,82.0,5.0
2,91.0,93.0,78.0,85.0,88.0,89.0,84.0,91.0,31.0,82.0,79.0,4.0
3,91.0,91.0,86.0,94.0,93.0,80.0,87.0,85.0,28.0,59.0,90.0,5.0
4,91.0,91.0,93.0,88.0,91.0,91.0,94.0,86.0,29.0,78.0,94.0,4.0


Scaling

In [146]:
from sklearn.preprocessing import StandardScaler

In [147]:
y = sports_data_21['overall']


In [148]:
x =sports_data_21.drop('overall',axis=1)

In [149]:

X= StandardScaler()

In [150]:
scaled = X.fit_transform(x)

In [151]:
sports_data_21=pd.DataFrame(scaled,columns=x.columns)

In [152]:
sports_data_21

,movement_reactions,passing,dribbling,mentality_composure,power_shot_power,attacking_short_passing,shooting,age,physic,mentality_vision,international_reputation
0,3.554438,3.493625,3.433013,3.137573,2.119026,2.216206,3.009606,1.655055,0.058838,3.000047,10.801035
1,3.664174,2.461857,2.800094,3.055051,2.719163,1.598583,3.085367,2.080838,1.363898,2.052781,10.801035
2,3.444701,2.152326,2.378148,2.477402,2.344077,1.735833,2.933845,1.229273,1.907673,1.834181,8.037314
3,3.225227,2.977741,3.327526,2.890009,1.668922,1.941707,2.479280,0.590598,-0.593692,2.635714,10.801035
4,3.225227,3.699979,2.694607,2.724966,2.494112,2.422080,2.555040,0.803490,1.472653,2.927180,8.037314
...,...,...,...,...,...,...,...,...,...,...,...
18939,-1.493455,-3.212867,-3.740069,-1.483625,-1.706851,-1.901280,-2.217897,-0.899642,-1.789997,-2.100615,-0.253847
18940,-1.273981,-0.839801,-1.630339,-1.896232,-1.481799,-0.254285,-1.536049,-0.899642,-1.789997,-0.060350,-0.253847
18941,-1.932402,-0.839801,-1.735826,-1.896232,-0.656610,-0.185661,-1.157245,0.590598,-0.593692,-0.716150,-0.253847
18942,-0.944771,-1.768392,-0.997420,-1.071018,-0.581593,-0.940533,-0.475396,-1.751207,-2.660037,-0.351817,-0.253847


In [153]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


Training Random Forest Regressor

In [154]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [155]:
rfm = RandomForestRegressor(n_estimators=110, random_state=45)
rfm.fit(X_train, y_train)


RandomForestRegressor(n_estimators=110, random_state=45)

In [156]:
scores = cross_val_score(rfm, X_train, y_train, cv=10)
print("Cross-Validation Scores:", scores)


Cross-Validation Scores: [0.91019605 0.91333297 0.91146354 0.90950853 0.91157812 0.91854096
 0.91157107 0.90158928 0.91404077 0.90535018]


In [157]:
y_pred = rfm.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 4.253497510191529


In [158]:
mae = print(mean_absolute_error(y_test, y_pred))

1.5773962906979533


Training with Decision Tree Classifier

In [159]:
from sklearn.tree import DecisionTreeRegressor

In [160]:
dtree = DecisionTreeRegressor(max_depth=15, min_samples_split=10)

In [161]:
dtree.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=15, min_samples_split=10)

In [162]:
y_pred = dtree.predict(X_test)

In [163]:
scores = cross_val_score(dtree, X_train, y_train, cv=10)
print("Cross-Validation Scores:", scores)


Cross-Validation Scores: [0.84354912 0.85602374 0.84611841 0.83437778 0.84382867 0.84785344
 0.84660617 0.83333594 0.85061534 0.84146003]


In [164]:
mae = print(mean_absolute_error(y_test, y_pred))

2.05731169284377


In [165]:
from sklearn.metrics import accuracy_score,f1_score,make_scorer

Training using Gradient Boost


In [166]:
from sklearn.ensemble import GradientBoostingRegressor

In [167]:
grad=GradientBoostingRegressor( n_estimators=100, learning_rate=0.1)

In [168]:
grad.fit(X_train,y_train)

GradientBoostingRegressor()

In [169]:
y_pred=grad.predict(X_test)

In [170]:

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 4.850331012225649


In [171]:
scores = cross_val_score(grad, X_train, y_train, cv=10)
print("Cross-Validation Scores:", scores)


Cross-Validation Scores: [0.90354792 0.90138681 0.89743212 0.8942029  0.90290018 0.90545881
 0.89742648 0.89622776 0.90891441 0.89829897]


In [172]:
mae = print(mean_absolute_error(y_test, y_pred))

1.6998522348669614


Testing with KNeighborsClassifier and SVC

In [173]:
m_ae = print(mean_absolute_error(y_test, y_pred))

1.6998522348669614


Voting Regressor

In [174]:
from sklearn.ensemble import VotingRegressor

In [175]:
# Create a VotingRegressor instance
voting_regressor = VotingRegressor(
    estimators=[
        ('gradboost', grad),
        ('rf', rfm),
        ('decision tree', dtree)
    ],
#  n_jobs=-1,  # Parallelize the training if available
#     weights=[1, 1, 1]  # Use 'hard' or 'soft' for aggregation
)


In [176]:
for model in (grad, rfm,dtree,voting_regressor):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)

    # Print the model's name and MAE
    print(model.__class__.__name__, "MAE:", mae)

GradientBoostingRegressor MAE: 1.6999322158839372
RandomForestRegressor MAE: 1.5773962906979533
DecisionTreeRegressor MAE: 2.044723783786937
VotingRegressor MAE: 1.6488228427190377


Optimization Of Models

In [177]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


In [178]:
cv = KFold(n_splits=6)
# Define the parameter grid
PARAMETERS = {
    "max_depth": [2, 5, 6, 12],
    "min_samples_split": [2, 5, 7,10],
    "n_estimators": [100,120, 150]
}




In [179]:
def custom_scorer(y_true, y_pred):
    return -mean_absolute_error(y_true, y_pred)  # Using negative MAE for GridSearchCV


In [180]:
rf_regressor = RandomForestRegressor(n_jobs=-1)
#Define the scoring function for regression (e.g., mean squared error)
scoring = make_scorer(mean_squared_error, greater_is_better=False)


In [181]:
# Create a GridSearchCV instance
model_gs = GridSearchCV(
    rf_regressor,
    param_grid=PARAMETERS,
    cv=cv,
    scoring=scoring,
    verbose=0
)


In [182]:
# Fit the model to the training data
model_gs.fit(X_train, y_train)

# Make predictions
y_pred = model_gs.predict(X_test)

Testing with Players 22

In [183]:
sports_data_22=pd.read_csv('/content/drive/MyDrive/MIDSEM PROJECT/players_22.csv')#train

<ipython-input-183-8692f08a1891>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  sports_data_22=pd.read_csv('/content/drive/MyDrive/MIDSEM PROJECT/players_22.csv')#train


In [184]:
threshold = 0.30 * len(sports_data_22)

# Identify columns with 30% or more NaN values
columns_with_nan = sports_data_22.columns[sports_data_22.isna().sum() >= threshold]

# Drop the identified columns
sports_data_22.drop(columns=columns_with_nan, inplace=True)

In [185]:
num_columns = sports_data_22.select_dtypes(include=['int64', 'float64'])

In [186]:
obj_columns = sports_data_22.select_dtypes(include=['object']).columns

In [187]:
for column in obj_columns:
    sports_data_22[column] = pd.factorize(sports_data_22[column])[0]


In [188]:

# Create a SimpleImputer instance with your chosen strategy
imputer = SimpleImputer(strategy='mean')

# Iterate over columns and impute
for column in sports_data_22:
    sports_data_22[column] = imputer.fit_transform(sports_data_22[[column]])


In [191]:
#We picked the top 12
selected_data =['overall','movement_reactions','passing','dribbling','mentality_composure','power_shot_power','attacking_short_passing','shooting','age','physic','mentality_vision','international_reputation']

In [192]:
sports_data_22 = sports_data_22[selected_data]

In [193]:
y_new= sports_data_22['overall']


In [194]:
x_new = sports_data_22.drop('overall',axis=1)

In [195]:

sc= StandardScaler()

In [196]:
scaled_new = sc.fit_transform(x_new)

In [197]:
sports_data_22=pd.DataFrame(scaled_new,columns=x_new.columns)

Testing with random forest regressor

In [198]:
# Now you can make predictions on the imputed data
y_pred_new = rfm.predict(x_new)


In [199]:
maeee = print(mean_absolute_error(y_new, y_pred_new))

1.4195086684717122


Testing with Decision Tree Regressor

In [200]:
# Now you can make predictions on the imputed data
y_pred_new = dtree.predict(x_new)


In [201]:
maee = print(mean_absolute_error(y_new, y_pred_new))

1.9220151386570443


Testing with Gradient Boost Regressor

In [202]:
# Now you can make predictions on the imputed data
y_pred_new = grad.predict(x_new)


In [203]:
maeee = print(mean_absolute_error(y_new, y_pred_new))

1.6635872001341772


Saving the Random Forest Regressor

In [ ]:
import pickle
filename = 'finalized_best_model.pkl'
pickle.dump(rfm, open(filename, 'wb'))
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X_test)
print(y_pred)

In [ ]:
# import pickle

# # Assuming you have a trained model named 'model'
# best_model_path = '/content/my_model.pkl'  # Specify the file path where you want to save the model

# # Save the model to a file
# with open(best_model_path, 'wb') as file:
#     pickle.dump(rfm, file)


In [ ]:
# from google.colab import files

# # Download the saved model
# files.download(best_model_path)
